In [ ]:
#Chinmay Bake Machine Learning Assignment3

In [91]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.classification import LogisticRegression
spark=SparkSession.builder.appName('Logistic_Regression').getOrCreate()

In [95]:
main_DF=spark.read.csv(r'D:\train.csv',inferSchema=True,header=True)

In [98]:
#First, determine if there are any missing values and if so, how many, for each variable, including the output variable, e.g. for variable age

main_DF.where(main_DF['Age'].isNull()).count()

177

In [99]:
main_DF.where(main_DF['Pclass'].isNull()).count()

0

In [100]:
main_DF.where(main_DF['Sex'].isNull()).count()

0

In [102]:
main_DF.where(main_DF['Embarked'].isNull()).count()

0

In [103]:
main_DF.where(main_DF['Survived'].isNull()).count()

'''Thus except for "Age" column, no other input or out variables have a missing value'''

0

In [105]:
# Next, you will have to find the average age for the data in the dataframe. You may to do this with the describe() function of the dataframe.

main_DF.describe().select('summary','Age','Pclass','Sex','Embarked').show()

+-------+------------------+------------------+------+--------+
|summary|               Age|            Pclass|   Sex|Embarked|
+-------+------------------+------------------+------+--------+
|  count|               714|               891|   891|     891|
|   mean| 29.69911764705882| 2.308641975308642|  null|    null|
| stddev|14.526497332334035|0.8360712409770491|  null|    null|
|    min|              0.42|                 1|female|       C|
|    max|              80.0|                 3|  male|       S|
+-------+------------------+------------------+------+--------+



In [106]:
#Finally, you will need to replace the null value for age with the average value. 

main_DF= main_DF.fillna({'Age':29.70})

In [107]:
main_DF.where(main_DF['Age'].isNull()).count()

0

In [108]:
# 1) What is the shape of the data contained in training.csv?

print((main_DF.count(),len(main_DF.columns)))

(891, 12)


In [110]:
# 2) What features (or attributes) are recorded for each passenger in training.csv?

main_DF.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [111]:
# 3) Provide a schema of the columns to be included in your model for this assignment.  Comment on columns that may require transformation(s). 

main_DF.printSchema()

'''Columns "Sex" and "Embarked" do not have a numeric data type and are categorical. For a Machine Learning 
model to process these variables, they need to be converted to a numeric data type. For this purpose, we utilise 
the concept of Dummy Variables where each category in the variable would represented by a dummy numeric variable.'''



root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = false)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [137]:
# 4) Comment on the balance of data in training.csv with regards to each input variable as well as your target variable. 
#Support your comments with appropriate statistics. 

main_DF.groupBy('Survived').count().show()

'''Less than 50% passengers survived the voyage. Hence we have more data with passangers who did not survived 
than the passangers who did.'''

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



'Less than 50% passengers survived the voyage.'

In [140]:
#4) 

main_DF.groupBy('Survived').mean().show()

'''Average age of the passangers who survived and the ones who did not is fairly similar.'''

+--------+------------------+-------------+------------------+------------------+-------------------+------------------+------------------+
|Survived|  avg(PassengerId)|avg(Survived)|       avg(Pclass)|          avg(Age)|         avg(SibSp)|        avg(Parch)|         avg(Fare)|
+--------+------------------+-------------+------------------+------------------+-------------------+------------------+------------------+
|       1|444.36842105263156|          1.0|1.9502923976608186|28.549912280701747|0.47368421052631576|0.4649122807017544| 48.39540760233917|
|       0| 447.0163934426229|          0.0|2.5318761384335153| 30.41530054644817| 0.5537340619307832|0.3296903460837887|22.117886885245877|
+--------+------------------+-------------+------------------+------------------+-------------------+------------------+------------------+



In [130]:
#4) 

main_DF.groupBy('Pclass').count().show()

'''We observe that the majority of passengers belonged to third class, followed by the ones who opted 
first class and the lastly second class. Hence there is a substantial imbalance in the count within the categories.'''

+------+-----+
|Pclass|count|
+------+-----+
|     1|  216|
|     3|  491|
|     2|  184|
+------+-----+



In [39]:
#4) 

df.groupBy('Sex').count().show()

'''There were more males than females on the voyage.'''

+------+-----+
|   Sex|count|
+------+-----+
|female|  314|
|  male|  577|
+------+-----+



In [41]:
#4) 

df.groupBy('Embarked').count().show()

'''The data is heavily biased towards Southampton as the port of embarkation.'''

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   78|
|       C|  168|
|       S|  645|
+--------+-----+



In [48]:
#4) 

df.groupBy('Sex').mean().show()

'''The average survival value for females is much closer to 1 than that of the males.
This possibly implies that more females managed surviving the disaster.'''

+------+------------------+-------------------+-----------------+------------------+-------------------+-------------------+------------------+
|   Sex|  avg(PassengerId)|      avg(Survived)|      avg(Pclass)|          avg(Age)|         avg(SibSp)|         avg(Parch)|         avg(Fare)|
+------+------------------+-------------------+-----------------+------------------+-------------------+-------------------+------------------+
|female|431.02866242038215| 0.7420382165605095|2.159235668789809|28.216878980891703| 0.6942675159235668| 0.6496815286624203| 44.47981783439487|
|  male| 454.1473136915078|0.18890814558058924|2.389948006932409|  30.5060138648181|0.42980935875216636|0.23570190641247835|25.523893414211418|
+------+------------------+-------------------+-----------------+------------------+-------------------+-------------------+------------------+



In [145]:
#5) Perform the transformations, if any, identified in step # 3.
#Perform feature engineering if and where needed, including Vectorization of relevant input variables. 

gender_indexer =StringIndexer(inputCol="Sex", outputCol="Gen_Numb").fit(main_DF)
gender_DF = gender_indexer.transform(main_DF)

In [146]:
embarked_indexer =StringIndexer(inputCol="Embarked", outputCol="Emb_Numb").fit(gender_DF)
embarked_DF = embarked_indexer.transform(gender_DF)

In [147]:
gender_encoder=OneHotEncoder(inputCol="Gen_Numb", outputCol="Gender_Dummy_Vector")
GenderEncoded_DF=gender_encoder.transform(embarked_DF)

In [148]:
embarked_encoder=OneHotEncoder(inputCol="Emb_Numb", outputCol="Embarked_Dummy_Vector")
EmbarkedEncoded_DF=embarked_encoder.transform(GenderEncoded_DF)

In [151]:
Assember_DF = VectorAssembler(inputCols=['Age','Pclass','Gender_Dummy_Vector','Embarked_Dummy_Vector'], outputCol="features")
Vector_DF = df_assembler.transform(EmbarkedEncoded_DF)

In [152]:
# Provide a printout of the schema of your feature-engineered data.
Vector_DF.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = false)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Gen_Num: double (nullable = false)
 |-- Emb_Num: double (nullable = false)
 |-- Gen_Numb: double (nullable = false)
 |-- Emb_Numb: double (nullable = false)
 |-- Gender_Dummy_Vector: vector (nullable = true)
 |-- Embarked_Dummy_Vector: vector (nullable = true)
 |-- features: vector (nullable = true)



In [155]:
# 6) To train and then test your model, split the data from training.csv into training and test datasets using an 80/20 split. 

Model_DF=Vector_DF.select(['features','Survived'])
train_DF,test_DF=Model_DF.randomSplit([0.80,0.20])

In [158]:
print("Training data count:",train_DF.count()," ","Testing data count:",test_DF.count())

Training data count: 705   Testing data count: 186


In [159]:
train_DF.groupBy('Survived').count().show()

#balance of data
'''We have more data points with target value 0 than with 1.'''

#Balance of target classes
'''The ratio of count of 1 to 0 in the overall dataset is 0.62 and in the training dataset is 0.64
Therefore equivalent proportions of data is included in the training dataset and it correctly represents the overall 
data'''

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  276|
|       0|  429|
+--------+-----+



In [160]:
test_DF.groupBy('Survived').count().show()

'''The test data set has a slight imbalance in the proportion of the target variables when compared with the overall data but 
the data is better distributed within the binary categories.'''

+--------+-----+
|Survived|count|
+--------+-----+
|       1|   66|
|       0|  120|
+--------+-----+



In [163]:
# 7) Build and train the Logistic Regression model. Generate a list of predictions for passengers survival status (survival = 1) based on the trained model. 
# Display actual, predicted, and probability values for the first 10 rows only

log_reg=LogisticRegression(labelCol='Survived').fit(train_DF)
Training_Results=log_reg.evaluate(training_df).predictions
Training_Results.select('Survived','prediction','probability').show(10,False)

+--------+----------+----------------------------------------+
|Survived|prediction|probability                             |
+--------+----------+----------------------------------------+
|1       |1.0       |[0.25581765877814716,0.7441823412218529]|
|1       |1.0       |[0.26212697221755493,0.7378730277824451]|
|1       |1.0       |[0.26212697221755493,0.7378730277824451]|
|0       |1.0       |[0.27504279752519617,0.7249572024748039]|
|0       |1.0       |[0.2951397358096188,0.7048602641903812] |
|1       |1.0       |[0.3020252651677627,0.6979747348322373] |
|1       |1.0       |[0.13957760734216204,0.860422392657838] |
|0       |1.0       |[0.35789587102105347,0.6421041289789464]|
|0       |1.0       |[0.35789587102105347,0.6421041289789464]|
|0       |1.0       |[0.35789587102105347,0.6421041289789464]|
+--------+----------+----------------------------------------+
only showing top 10 rows



In [ ]:
#Based on these results, comment on the performance of the model? Is the model predicting likelihood of survival with high probability?

'''The first value in the probability represents 0(did not survive) and the second one represents the 
one who survived; we could observe that for the first 10 records displayed, all the second probability values
are greater than the first one. That is essentially the resaon why all the predicted values are equal to 1. 
 Based on evaluating these specific records, the model performance is pretty average. The model has predicted 
 half of the records correcly, while half of the predictions are False Positives.'''

In [164]:
# 8) Using the test data from the 80/20 split, evaluate the performance of your trained model. 

Testing_Results=log_reg.evaluate(test_DF).predictions

In [166]:
# Compute and show the values for Accuracy, Recall, Precision, and an F1 score. 

tp = Testing_Results[(Testing_Results.Survived == 1) & (Testing_Results.prediction == 1)].count()
tn = Testing_Results[(Testing_Results.Survived == 0) & (Testing_Results.prediction == 0)].count()
fp = Testing_Results[(Testing_Results.Survived == 0) & (Testing_Results.prediction == 1)].count()
fn = Testing_Results[(Testing_Results.Survived == 1) & (Testing_Results.prediction == 0)].count()

In [168]:
print("Accuracy:",float((tp+tn) /(tp+tn+fp+fn)))


Accuracy: 0.8064516129032258


In [169]:
print("Recall Rate:",float(tp)/(tp+fn))

Recall Rate: 0.7424242424242424


In [171]:
print("Precision:", float(tp) / (tp + fp))

Precision: 0.7205882352941176


In [178]:
precision=float(tp) / (tp + fp)
recall=float(tp)/(tp+fn)
print("F1 Score:",2*(precision*recall)/(precision+recall))

F1 Score: 0.7313432835820897


In [172]:
# Comment of general usefulness of the model in predicting the survival status of passengers given their age, gender, pclass and embarked values.

'''The model yeilds an accuracy of 80% which could be considered as quite good. Accuracy although is dependent on how 
balanced the input data is. Therefore we compute the precision of the model for better evauluation. Every time 
the model makes a prediction, 74% of the Postives (survived) are correctly predicted. We have the recall rate and precision 
in the similar range; meaning that our input test data is pretty well balanced.'''

'The model yeilds an accuracy of 80% which could be considered as quite good. Accuracy although\nis dependent on how balanced the input data is. Therefore we compute the recall rate of the model.\nEvery time the model makes a prediction, 74% of the Postives would be correctly predicted.'

In [176]:
#Calculate ROC 

#a.obtain a summary of the model trained with training dataset. 
#Assuming that the instance of the logistic regression model fitted with training data is called ‘log_reg

training_summary = log_reg.summary

In [177]:
#b From the training_summary, obtain and print the AUC value

print("areaUnderROC: " + str(training_summary.areaUnderROC))

areaUnderROC: 0.8500853011722578


In [180]:
#Accuracy, Precision, Recall and F1 obtained from summary

print("Accuracy:",training_summary.accuracy,"Recall:",training_summary.weightedRecall,
      "Precision:",training_summary.weightedPrecision,"F1 Score:",training_summary.weightedFMeasure())

Accuracy: 0.798581560283688 Recall: 0.7985815602836879 Precision: 0.7969395642799602 F1 Score: 0.7970895590646272


In [181]:
# Using the AUC, Accuracy, Recall, Precision and F1 score calculated in this step, please discuss the performance of the trained model. 

'''Comparing the evaluations for the test and train data, we observe a marginal difference in the accuracy of the model. 
All in all, we can observe that the model better predicts with training data with a higher F1 score and better precision value.'''

'Comparing the evaluations for the test and train data, we observe a marginal difference in the accuracy of the model. \nAll in all, we can observed that the model better predicts with training data owing to higher F1 score and precision values'